# STEREO/SECCHI-EUVI

In [ ]:
ds_path = "./data/stereo/euvi"

start_date = "2011-01-01"
end_date = "2011-01-02"

## explore

In [ ]:
from pathlib import Path

wavelengths = [171, 195, 284, 304]
dirs = [str(wl) for wl in wavelengths]

[(Path(ds_path) / "a" / wl).mkdir(parents=True, exist_ok=True) for wl in dirs]
[(Path(ds_path) / "b" / wl).mkdir(parents=True, exist_ok=True) for wl in dirs]

[None, None, None, None]

In [ ]:
from datetime import datetime, timedelta

t_start = datetime.strptime(start_date, "%Y-%m-%d")
t_end = datetime.strptime(end_date, "%Y-%m-%d")
td = timedelta(hours=12)
date_list = [t_start + i * td for i in range((t_end - t_start) // td)]

In [ ]:
d = date_list[0]

In [ ]:
d

datetime.datetime(2011, 1, 1, 0, 0)

In [ ]:
root = "https://stereo-ssc.nascom.nasa.gov/data/ins_data/secchi/L0_YMD/"

In [ ]:
root_a = root + "a/img/euvi/"
root_b = root + "b/img/euvi/"

In [ ]:
date = datetime.strftime(d, "%Y/%m/%d/")
date

'2011/01/01/'

In [ ]:
stereo_a_url = root_a + date
stereo_b_url = root_b + date

In [ ]:
stereo_a_url

'https://stereo-ssc.nascom.nasa.gov/data/ins_data/secchi/L0_YMD/a/img/euvi/2011/01/01/'

In [ ]:
stereo_b_url

'https://stereo-ssc.nascom.nasa.gov/data/ins_data/secchi/L0_YMD/b/img/euvi/2011/01/01/'

In [ ]:
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup

def get_bs(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        print(e)
    except URLError as e:
        print("The server could not be found!")
    else:
        bs = BeautifulSoup(html.read(), 'html.parser')
        return bs

In [ ]:
bs = get_bs(stereo_a_url)

In [ ]:
link_list = bs.find_all('a')

In [ ]:
len(link_list)

606

In [ ]:
link_list[:10]

[<a href="?C=N;O=D">Name</a>,
 <a href="?C=M;O=A">Last modified</a>,
 <a href="?C=S;O=A">Size</a>,
 <a href="/data/ins_data/secchi/L0_YMD/a/img/euvi/2011/01/">Parent Directory</a>,
 <a href="20110101_000530_n4euA.fts">20110101_000530_n4euA.fts</a>,
 <a href="20110101_000615_n4euA.fts">20110101_000615_n4euA.fts</a>,
 <a href="20110101_001030_n4euA.fts">20110101_001030_n4euA.fts</a>,
 <a href="20110101_001400_n4euA.fts">20110101_001400_n4euA.fts</a>,
 <a href="20110101_001530_n4euA.fts">20110101_001530_n4euA.fts</a>,
 <a href="20110101_001615_n4euA.fts">20110101_001615_n4euA.fts</a>]

In [ ]:
import re
fts_re = re.compile(datetime.strftime(d, "%Y%m%d") + ".*.fts")
fts_re

re.compile(r'20110101.*.fts', re.UNICODE)

In [ ]:
fts_list = bs.find_all('a', {'href': fts_re})

In [ ]:
len(fts_list)

602

In [ ]:
f = fts_list[0]
f

<a href="20110101_000530_n4euA.fts">20110101_000530_n4euA.fts</a>

In [ ]:
for i, f in enumerate(fts_list):
    obstime = datetime.strptime(f.get('href')[:15], "%Y%m%d_%H%M%S")
    if obstime.hour != d.hour:
        break

In [ ]:
def get_idx(fts_list, date):
    # find the first index of the fts file that has the same hour as the date
    for i, f in enumerate(fts_list):
        obstime = datetime.strptime(f.get('href')[:15], "%Y%m%d_%H%M%S")
        if obstime.hour == date.hour:
            return i

In [ ]:
i = get_idx(fts_list, d)
fts_list[i]

<a href="20110101_000530_n4euA.fts">20110101_000530_n4euA.fts</a>

In [ ]:
dd = date_list[1]
print(dd)
i = get_idx(fts_list, dd)
fts_list[i]

2011-01-01 12:00:00


<a href="20110101_120030_n4euA.fts">20110101_120030_n4euA.fts</a>

In [ ]:
from astropy.io import fits

In [ ]:
f.get('href')

'20110101_010030_n4euA.fts'

In [ ]:
url = stereo_a_url + f.get('href')
header = fits.getheader(url)
header['DATE-OBS']

'2011-01-01T01:00:30.007'

In [ ]:
# Create url list until all possible wavelengths are found.
data = []
possible_values = {171, 195, 284, 304}
seen_values = set()

idx = get_idx(fts_list, dd)
for f in fts_list[idx:]:
    url = stereo_a_url + f.get('href')
    # hdul = fits.open(url)
    # header = hdul[0].header
    # hdul.close()
    header = fits.getheader(url)
    if header['NAXIS1'] < 2048 or header['NAXIS2'] < 2048 or header['NMISSING'] != 0:
        print("Invalid file:", f.get('href'))
        continue

    info = {}
    info['obstime'] = datetime.strptime(f.get('href')[:15], "%Y%m%d_%H%M%S")
    info['wavelength'] = header['WAVELNTH']
    info['source'] = "a"
    info['url'] = url
    data.append(info)

    seen_values.add(int(header['WAVELNTH']))
    if seen_values == possible_values:
        break

In [ ]:
len(data)

8

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(data)
df

,obstime,wavelength,source,url
0,2011-01-01 12:00:30,195,a,https://stereo-ssc.nascom.nasa.gov/data/ins_da...
1,2011-01-01 12:05:30,195,a,https://stereo-ssc.nascom.nasa.gov/data/ins_da...
2,2011-01-01 12:06:15,304,a,https://stereo-ssc.nascom.nasa.gov/data/ins_da...
3,2011-01-01 12:10:30,195,a,https://stereo-ssc.nascom.nasa.gov/data/ins_da...
4,2011-01-01 12:14:00,171,a,https://stereo-ssc.nascom.nasa.gov/data/ins_da...
5,2011-01-01 12:15:30,195,a,https://stereo-ssc.nascom.nasa.gov/data/ins_da...
6,2011-01-01 12:16:15,304,a,https://stereo-ssc.nascom.nasa.gov/data/ins_da...
7,2011-01-01 12:16:30,284,a,https://stereo-ssc.nascom.nasa.gov/data/ins_da...


In [ ]:
df_171 = df[df['wavelength'] == 171].sort_values(by='obstime').reset_index(drop=True)
df_195 = df[df['wavelength'] == 195].sort_values(by='obstime').reset_index(drop=True)
df_284 = df[df['wavelength'] == 284].sort_values(by='obstime').reset_index(drop=True)
df_304 = df[df['wavelength'] == 304].sort_values(by='obstime').reset_index(drop=True)

In [ ]:
df_171

,obstime,wavelength,source,url
0,2011-01-01 12:14:00,171,a,https://stereo-ssc.nascom.nasa.gov/data/ins_da...


In [ ]:
df_195

,obstime,wavelength,source,url
0,2011-01-01 12:00:30,195,a,https://stereo-ssc.nascom.nasa.gov/data/ins_da...
1,2011-01-01 12:05:30,195,a,https://stereo-ssc.nascom.nasa.gov/data/ins_da...
2,2011-01-01 12:10:30,195,a,https://stereo-ssc.nascom.nasa.gov/data/ins_da...
3,2011-01-01 12:15:30,195,a,https://stereo-ssc.nascom.nasa.gov/data/ins_da...


In [ ]:
df_284

,obstime,wavelength,source,url
0,2011-01-01 12:16:30,284,a,https://stereo-ssc.nascom.nasa.gov/data/ins_da...


In [ ]:
df_304

,obstime,wavelength,source,url
0,2011-01-01 12:06:15,304,a,https://stereo-ssc.nascom.nasa.gov/data/ins_da...
1,2011-01-01 12:16:15,304,a,https://stereo-ssc.nascom.nasa.gov/data/ins_da...


In [ ]:
sample = df_171.iloc[0]
sample.obstime

Timestamp('2011-01-01 12:14:00')

In [ ]:
def round_hour(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
            + timedelta(hours=t.minute // 30))

In [ ]:
round_hour(sample.obstime)

Timestamp('2011-01-01 12:00:00')

In [ ]:
Path(ds_path) / sample.source / str(sample.wavelength)

PosixPath('data/stereo/euvi/a/171')